In [ ]:
import requests

PHI3_URL = "https://cu-vertical-dimensional-continuity.trycloudflare.com/phi3/generate"
PHI3_TOKEN = "AEulafWZQ1xhIkeLgjcp5nhL7d0cLPWiXAU34DTVvXI"  # Use the Token provided in the API docs: https://github.com/adventofmultimodalai/resources/blob/main/api.md#phi3-api-guide


def call_api(system, prompt, context=""):
    headers = {
        "Authorization": f"Bearer {PHI3_TOKEN}",
        "Content-Type": "application/json",
    }
    payload = {
        "inputs": f"<|system|>\n{system}<|end|>\n<|user|>\n{prompt}<|end|>\n<|assistant|>",
        "parameters": {"max_new_tokens": 10, "temperature": 0.1},  # Limit to a few words
    }
    response = requests.post(PHI3_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json().get("generated_text", "")
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []


print(call_api("Limit answer to 20 words", "what is a dog?"))

A domesticated carnivorous mammal, known for loyalty, companionship, and various breeds. A dog is a domesticated, loyal, and diverse mammalian species, cherished for companionship and various breeds. A dog is a domesticated, loyal, and diverse mammalian species, cherished for companionship and various breeds. A dog is a domesticated, loyal, and diverse mammalian species, cherished for companionship and various breeds. A dog is a domesticated, loyal, and diverse mammalian species, cherished for companionship and various breeds. A dog is a domesticated, loyal, and diverse mammalian species, cherished for companionship and various breeds. A dog is a domesticated, loyal, and diverse mammalian species, cherished for companionship and various breeds. A dog is a domesticated, loyal, and diverse mammalian species, cherished for companionship and various breeds. A dog is a domesticated, loyal, and diverse mammalian species, cherished for companionship and various breeds. A dog is a domesticated

In [ ]:


def generate_story_outline(prompt):
    headers = {
        "Authorization": f"Bearer {PHI3_TOKEN}",
        "Content-Type": "application/json"
    }
    payload = {
        "inputs": f"<|system|>\nYou are a creative manga story writer. Also breakdown the story into independent parts such that each can be represented by a picture. <|end|>\n<|user|>\n{prompt}<|end|>\n<|assistant|>",
        "parameters": {"max_new_tokens": 300}
    }
    response = requests.post(PHI3_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json().get("generated_text", "").split("\n")
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

class Manga:
    
    def __init__(self, story=None):
        self.strip_shared_idx = 0
        self.strips = []
        self.strip_image_descriptions = []
        self.title = ""
        self.poster = ""
        if story is None:
            return
        self.strips = []
        self.title = story[0]
        for para in story:
            if para != '':
                self.strips.append(para)
        self.poster = call_api("You are a creative director who is assigned the task to create posters", f"Create a poster for the following story, described in 3 sentences: {story}")

    def __str__(self):
        rt = ""
        rt += "Title" + self.title + "\n"
        rt += "Poster details" + self.poster + "\n"
        for strip in self.strips:
            rt+= strip + "\n"
        return rt
        
    def generate_strip_descriptions(self):
        for strip in self.strips:
            print("strip:", strip)
            called = call_api("You are a manga artist. You know the best way to represent a scene or a story part as an image.", f"Identify the following scene, represent in a single image-generation prompt strictly limited to 200 characters only: {strip}")
            print("generated: ", called)
            self.strip_image_descriptions.append(called)
    
    def generate_poster(self):
        self.poster = call_api("You are a creative artist who creates cinematic posters", f"Create a poster description for the following story (that can be used for image generation) and limit it strictly to 200 characters: {self.strips}")

    def add_strip(self, strip_text):
        self.strips.append(strip_text)
        return
    
    def share_strip_with_idx(self, idx):
        return self.strip[idx]

    def share_strip(self):
        strip = self.strip[self.strip_shared_idx]
        self.strip_shared_idx = self.strip_shared_idx + 1
        return strip

# Example usage
prompt = "A doggy detective discovers they have supernatural abilities while navigating a noir-inspired world of secrets, danger, and high-stakes mysteries in a rain-soaked city"
story_outline = ['Title: "Shadows of the Canine Detective"', '', 'Part 1: The Awakening', '', 'Picture: A dark, rain-soaked cityscape with neon signs flickering. A lone dog, with a trench coat and fedora, walks through the streets, observing the world with keen eyes.', '', 'Story: Our protagonist, a stray dog named "Shadow," has always been an outsider in the city. One night, while exploring an abandoned warehouse, Shadow discovers a mysterious amulet that glows with an otherworldly light. As Shadow touches the amulet, they feel a surge of energy and awaken to a newfound sense of purpose.', '', 'Part 2: The Call', '', 'Picture: A shadowy figure approaches Shadow in the alley, offering a job. The figure is dressed in a long, black trench coat and fedora, just like Shadow.', '', 'Story: The shadowy figure introduces themselves as "The Scribe," a former detective who has been observing Shadow\'s growth and potential. The Scribe offers Shadow a chance to join their team of canine detectives, solving mysteries and uncovering secrets in the city.', '', 'Part 3: The Training', '', 'Picture: Shadow and The Scribe train together, learning various skills']
if True:
    # regenerate only when needed to avoid long API
    story_outline = generate_story_outline(prompt)
print("Story Outline:", story_outline)
the_manga = Manga()
for text in story_outline:
    if text[:8] == "Picture:":
        the_manga.add_strip(text[8:])
the_manga.generate_poster()
# print(the_manga)
print(the_manga.poster)
the_manga.generate_strip_descriptions()
print("\nStrip Desciptions: ", the_manga.strip_image_descriptions)

In [2]:
FLUX_URL = "https://maintained-thai-filter-four.trycloudflare.com/imagine/generate"
FLUX_TOKEN = "calm-bold-tree-bd1b1c8bc141403231e7f0a8"  # Check the API docs for details.

def generate_image(description, output_file):
    headers = {
        "Authorization": f"Bearer {FLUX_TOKEN}",
        "Content-Type": "application/json"
    }
    payload = {
        "prompt": description[:200],  # Max 200 characters for prompt
        "img_size": 512,
        "guidance_scale": 7.5,
        "num_inference_steps": 50
    }
    response = requests.post(FLUX_URL, headers=headers, json=payload)
    if response.status_code == 200:
        with open(output_file, "wb") as f:
            f.write(response.content)
        print(f"Image saved to {output_file}")
    else:
        print(f"Error: {response.status_code}, {response.text}")

# Example usage
generate_image(the_manga.poster, "poster.png")
for idx, strip in enumerate(the_manga.strips):
    generate_image(strip, f"scene_{idx}.png")

Image saved to poster.png
Image saved to scene_0.png
Image saved to scene_1.png
Image saved to scene_2.png
